In [1]:
# requests>=2.31.0
# websockets>=12.0

import requests
import json

# Configuration
BASE_URL = "http://localhost:8000"
WS_BASE_URL = "ws://localhost:8000"


# Sample CSV content
CSV_CONTENT = """name,address,phone
General Hospital,123 Main St,555-1234
City Medical Center,456 Oak Ave,555-5678
Community Hospital,789 Pine Rd,555-9012
Regional Health,321 Elm St,555-3456
Memorial Hospital,654 Maple Dr,555-7890
Regional Health,322 Elm St,555-3456
Memorial Hospital,653 Maple Dr,
Community Hospital,789 Pine Rd,555-9012
Regional Health,323 Elm St,555-3456
Memorial Hospital,655 Maple Dr,555-7890
Regional Health,324 Elm St,555-3456
Memorial Hospital,656 Maple Dr,"""


In [2]:
# Create a file-like object from CSV content
files = {
    'file': ('hospitals.csv', CSV_CONTENT.encode('utf-8'), 'text/csv')
}

try:
    response = requests.post(f"{BASE_URL}/api/v1/hospital/bulk", files=files)
    res_json = response.json()
except Exception as e:
    res_json = {}
    print(f"Error: {e}")
res_json

{'batch_id': '252b7889-c199-4882-917b-d7789bbfa239',
 'status': 'processing',
 'duplicates_removed': 1,
 'total_hospitals': 11,
 'message': 'Batch processing started',
 'duplicate_hospitals': [{'row': 8,
   'name': 'Community Hospital',
   'address': '789 Pine Rd',
   'phone': '555-9012'}]}

In [3]:
res_json["batch_id"]

'252b7889-c199-4882-917b-d7789bbfa239'

In [4]:
res_json

{'batch_id': '252b7889-c199-4882-917b-d7789bbfa239',
 'status': 'processing',
 'duplicates_removed': 1,
 'total_hospitals': 11,
 'message': 'Batch processing started',
 'duplicate_hospitals': [{'row': 8,
   'name': 'Community Hospital',
   'address': '789 Pine Rd',
   'phone': '555-9012'}]}

In [7]:
import asyncio
import websockets
batch_id = res_json["batch_id"]
uri = f"{WS_BASE_URL}/api/v1/hospital/status?batch_id={batch_id}"
    
try:
    async with websockets.connect(uri) as websocket:
        print("✅ Connected to WebSocket")
        print("📡 Monitoring status updates...\n")
        
        update_count = 0
        while True:
            try:
                message = await asyncio.wait_for(websocket.recv(), timeout=2.0)
                update_count += 1
                
                data = json.loads(message)
                
                if "error" in data:
                    print(f"Error: {data['error']}")
                    break
                print(data)
                    
            except asyncio.TimeoutError:
                continue
            except websockets.exceptions.ConnectionClosed:
                print("🔌 Connection closed")
                break
                
except Exception as e:
    print(f"Error: {repr(e)}")


✅ Connected to WebSocket
📡 Monitoring status updates...

{'batch_id': '252b7889-c199-4882-917b-d7789bbfa239', 'status': 'completed', 'total_hospitals': 11, 'processed_hospitals': 11, 'successful_hospitals': 11, 'failed_hospitals': 0, 'batch_activated': True, 'hospitals': [{'row': 1, 'hospital_id': 168, 'name': 'General Hospital', 'address': '123 Main St', 'phone': '555-1234', 'status': 'success', 'error': None}, {'row': 2, 'hospital_id': 169, 'name': 'City Medical Center', 'address': '456 Oak Ave', 'phone': '555-5678', 'status': 'success', 'error': None}, {'row': 3, 'hospital_id': 166, 'name': 'Community Hospital', 'address': '789 Pine Rd', 'phone': '555-9012', 'status': 'success', 'error': None}, {'row': 4, 'hospital_id': 167, 'name': 'Regional Health', 'address': '321 Elm St', 'phone': '555-3456', 'status': 'success', 'error': None}, {'row': 5, 'hospital_id': 171, 'name': 'Memorial Hospital', 'address': '654 Maple Dr', 'phone': '555-7890', 'status': 'success', 'error': None}, {'row':

In [7]:
batch_id = res_json["batch_id"]
payload = {
    "batch_id": batch_id
}
try:
    response = requests.post(
        f"{BASE_URL}/api/v1/hospital/retry",
        json=payload
    )
    res_json = response.json()
except Exception as e:
    res_json = {}
    print(f"Error: {e}")

In [8]:

# delete all entries
try:
    res = requests.get("https://hospital-directory.onrender.com/hospitals/")
    res_json = res.json()

    unique_ids = set()
    for hospital in res_json:
        if hospital["creation_batch_id"] not in unique_ids:
            unique_ids.add(hospital["creation_batch_id"])

    for id in unique_ids:
        del_res = requests.delete(f"https://hospital-directory.onrender.com/hospitals/batch/{id}")
        print(del_res.text)
except Exception as e:
    res = {}
    print(f"Error: {repr(e)}")



{"deleted_count":11,"message":"Deleted 11 hospital(s) with batch ID dd21f7c2-0b6b-468f-8c7a-e87da933cc4f"}
{"deleted_count":11,"message":"Deleted 11 hospital(s) with batch ID 9580f01e-8637-4c2f-bfae-9195b317fa7e"}
